This notebook generates a Jazz music piece by obtaining a dataset of 78 unique values, a value being equivalent to a note, comprising of pitch and duration. So there will be $m$ training examples with $T_x =30$ values. At each time step, the input is  a one-hot vector representing 78 values. Then learn an RNN model to generate sequences of values. 

* The model takes input X of shape $(m, T_x, 78)$ and labels Y of shape $(T_y, m, 78)$. 
* An LSTM is used with hidden states that have $n_{a} = 64$ dimensions.

<img src="images/music_generation.png" style="width:600;height:400px;">

In [8]:
from __future__ import print_function
import IPython
import sys
from music21 import *
import numpy as np

from grammar import *
from qa import *
from preprocess import * 
from musicutils import *
from musicdatautils import *

from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

In [9]:
X, Y, n_values, indices_values = load_music_utils()
print('number of training examples:', X.shape[0])
print('Tx (length of sequence):', X.shape[1])
print('total # of unique values:', n_values)
print('shape of X:', X.shape)
print('Shape of Y:', Y.shape)

number of training examples: 60
Tx (length of sequence): 30
total # of unique values: 78
shape of X: (60, 30, 78)
Shape of Y: (30, 60, 78)


In [10]:
n_a = 64
n_values = 78 # number of music values
reshapor = Reshape((1, n_values))                       
LSTM_cell = LSTM(n_a, return_state = True)         
densor = Dense(n_values, activation='softmax') 

In [11]:
def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras instance model with n_a activations
    """
    
    # Define the input layer and specify the shape
    X = Input(shape=(Tx, n_values))
    
    # Define the initial hidden state a0 and initial cell state c0
    # using `Input`
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    # Create empty list to append the outputs while you iterate
    outputs = list()
    
    for t in range(Tx):

        # Select the "t"th time step vector from X. 
        x = Lambda(lambda x : X[:,t,:]) (X)
        # Use reshapor to reshape x to be (1, n_values)
        x = reshapor(x)
        # Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(inputs=x, initial_state=[a, c])
        # Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # add the output to "outputs"
        outputs.append(out)
        
    # Create model instance
    model = Model(inputs=[X, a0, c0], outputs=outputs)
    
    return model

In [12]:
model = djmodel(Tx = 30 , n_a = 64, n_values = 78)

In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 78)       0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 78)           0           input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 78)        0           lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   
                                                                 lambda_3[0][0]                   
                                                                 lambda_4[0][0]                   
          

In [14]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# Initialize hidden state and cell state
m = 60
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

In [17]:
model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
60/60 [==============================] - 5s 88ms/step - loss: 125.8697 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0167 - dense_1_acc_1: 0.0667 - dense_1_acc_2: 0.0833 - dense_1_acc_3: 0.0333 - dense_1_acc_4: 0.0500 - dense_1_acc_5: 0.0667 - dense_1_acc_6: 0.0333 - dense_1_acc_7: 0.0667 - dense_1_acc_8: 0.0667 - dense_1_acc_9: 0.0667 - dense_1_acc_10: 0.1500 - dense_1_acc_11: 0.0500 - dense_1_acc_12: 0.0833 - dense_1_acc_13: 0.1333 - dense_1_acc_14: 0.0667 - dense_1_acc_15: 0.0833 - dense_1_acc_16: 0.0833 - dense_1_acc_17: 0.1000 - dense_1_acc_18: 0.0833 - dense_1_acc_19: 0.0167 - dense_1_acc_20: 0.1000 - dense_1_acc_21: 0.0833 - dense_1_acc_22: 0.0833 - dense_1_acc_23: 0.0500 - dense_1_acc_24: 0.0500 - dense_1_acc_25: 0.1667 - dense_1_acc_26: 0.0333 - dense_1_acc_27: 0.1167 - dense_1_acc_28: 0.0833 - dense_1_acc_29: 0.0000e+00                                           
Epoch 2/100
60/60 [==============================] - 0s 1ms/step - loss: 122.4213 - dense_1_loss: 0.0000e

60/60 [==============================] - 0s 1ms/step - loss: 94.6291 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.1000 - dense_1_acc_1: 0.1833 - dense_1_acc_2: 0.2000 - dense_1_acc_3: 0.1833 - dense_1_acc_4: 0.2167 - dense_1_acc_5: 0.1167 - dense_1_acc_6: 0.1833 - dense_1_acc_7: 0.2167 - dense_1_acc_8: 0.2333 - dense_1_acc_9: 0.1667 - dense_1_acc_10: 0.2500 - dense_1_acc_11: 0.1500 - dense_1_acc_12: 0.2167 - dense_1_acc_13: 0.2833 - dense_1_acc_14: 0.1667 - dense_1_acc_15: 0.1500 - dense_1_acc_16: 0.2667 - dense_1_acc_17: 0.1500 - dense_1_acc_18: 0.1500 - dense_1_acc_19: 0.1833 - dense_1_acc_20: 0.2000 - dense_1_acc_21: 0.1667 - dense_1_acc_22: 0.1167 - dense_1_acc_23: 0.1167 - dense_1_acc_24: 0.1000 - dense_1_acc_25: 0.2333 - dense_1_acc_26: 0.1000 - dense_1_acc_27: 0.1667 - dense_1_acc_28: 0.1167 - dense_1_acc_29: 0.0000e+00
Epoch 11/100
60/60 [==============================] - 0s 1ms/step - loss: 90.9993 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0667 - dense_1_acc_1: 0.1833 - dense

60/60 [==============================] - 0s 1ms/step - loss: 63.4621 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0667 - dense_1_acc_1: 0.2500 - dense_1_acc_2: 0.3000 - dense_1_acc_3: 0.2333 - dense_1_acc_4: 0.3000 - dense_1_acc_5: 0.3000 - dense_1_acc_6: 0.3500 - dense_1_acc_7: 0.4500 - dense_1_acc_8: 0.4667 - dense_1_acc_9: 0.4667 - dense_1_acc_10: 0.4000 - dense_1_acc_11: 0.3833 - dense_1_acc_12: 0.4667 - dense_1_acc_13: 0.4333 - dense_1_acc_14: 0.3667 - dense_1_acc_15: 0.3333 - dense_1_acc_16: 0.4500 - dense_1_acc_17: 0.4167 - dense_1_acc_18: 0.5000 - dense_1_acc_19: 0.4667 - dense_1_acc_20: 0.4333 - dense_1_acc_21: 0.4000 - dense_1_acc_22: 0.4833 - dense_1_acc_23: 0.3667 - dense_1_acc_24: 0.2833 - dense_1_acc_25: 0.6000 - dense_1_acc_26: 0.4667 - dense_1_acc_27: 0.4000 - dense_1_acc_28: 0.4833 - dense_1_acc_29: 0.0000e+00
Epoch 20/100
60/60 [==============================] - 0s 1ms/step - loss: 60.4122 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0667 - dense_1_acc_1: 0.2667 - dense

60/60 [==============================] - 0s 1ms/step - loss: 39.0771 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0333 - dense_1_acc_1: 0.2667 - dense_1_acc_2: 0.4500 - dense_1_acc_3: 0.3333 - dense_1_acc_4: 0.5333 - dense_1_acc_5: 0.6000 - dense_1_acc_6: 0.7833 - dense_1_acc_7: 0.8167 - dense_1_acc_8: 0.7833 - dense_1_acc_9: 0.9000 - dense_1_acc_10: 0.7667 - dense_1_acc_11: 0.9000 - dense_1_acc_12: 0.8833 - dense_1_acc_13: 0.9333 - dense_1_acc_14: 0.8667 - dense_1_acc_15: 0.8833 - dense_1_acc_16: 0.9167 - dense_1_acc_17: 0.9167 - dense_1_acc_18: 0.8500 - dense_1_acc_19: 0.8833 - dense_1_acc_20: 0.9000 - dense_1_acc_21: 0.9333 - dense_1_acc_22: 0.8500 - dense_1_acc_23: 0.8833 - dense_1_acc_24: 0.8333 - dense_1_acc_25: 0.9333 - dense_1_acc_26: 0.8667 - dense_1_acc_27: 0.8667 - dense_1_acc_28: 0.8833 - dense_1_acc_29: 0.0000e+00
Epoch 29/100
60/60 [==============================] - 0s 1ms/step - loss: 36.9225 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0333 - dense_1_acc_1: 0.2667 - dense

60/60 [==============================] - 0s 1ms/step - loss: 22.6050 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0333 - dense_1_acc_1: 0.3500 - dense_1_acc_2: 0.6000 - dense_1_acc_3: 0.6167 - dense_1_acc_4: 0.8333 - dense_1_acc_5: 0.9667 - dense_1_acc_6: 0.9333 - dense_1_acc_7: 0.9667 - dense_1_acc_8: 0.9667 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 0.9667 - dense_1_acc_12: 0.9833 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000 - dense_1_acc_16: 1.0000 - dense_1_acc_17: 1.0000 - dense_1_acc_18: 1.0000 - dense_1_acc_19: 1.0000 - dense_1_acc_20: 1.0000 - dense_1_acc_21: 1.0000 - dense_1_acc_22: 1.0000 - dense_1_acc_23: 1.0000 - dense_1_acc_24: 1.0000 - dense_1_acc_25: 1.0000 - dense_1_acc_26: 1.0000 - dense_1_acc_27: 0.9833 - dense_1_acc_28: 0.9833 - dense_1_acc_29: 0.0000e+00
Epoch 38/100
60/60 [==============================] - 0s 1ms/step - loss: 21.2758 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0333 - dense_1_acc_1: 0.3500 - dense

...

60/60 [==============================] - 0s 1ms/step - loss: 8.3973 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0333 - dense_1_acc_1: 0.5500 - dense_1_acc_2: 0.8667 - dense_1_acc_3: 0.9833 - dense_1_acc_4: 0.9833 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000 - dense_1_acc_16: 1.0000 - dense_1_acc_17: 1.0000 - dense_1_acc_18: 1.0000 - dense_1_acc_19: 1.0000 - dense_1_acc_20: 1.0000 - dense_1_acc_21: 1.0000 - dense_1_acc_22: 1.0000 - dense_1_acc_23: 1.0000 - dense_1_acc_24: 1.0000 - dense_1_acc_25: 1.0000 - dense_1_acc_26: 1.0000 - dense_1_acc_27: 1.0000 - dense_1_acc_28: 0.9833 - dense_1_acc_29: 0.0000e+00
Epoch 65/100
60/60 [==============================] - 0s 1ms/step - loss: 8.2670 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0333 - dense_1_acc_1: 0.5667 - dense_1

60/60 [==============================] - 0s 1ms/step - loss: 7.4308 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0333 - dense_1_acc_1: 0.5833 - dense_1_acc_2: 0.8833 - dense_1_acc_3: 0.9833 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000 - dense_1_acc_16: 1.0000 - dense_1_acc_17: 1.0000 - dense_1_acc_18: 1.0000 - dense_1_acc_19: 1.0000 - dense_1_acc_20: 1.0000 - dense_1_acc_21: 1.0000 - dense_1_acc_22: 1.0000 - dense_1_acc_23: 1.0000 - dense_1_acc_24: 1.0000 - dense_1_acc_25: 1.0000 - dense_1_acc_26: 1.0000 - dense_1_acc_27: 1.0000 - dense_1_acc_28: 1.0000 - dense_1_acc_29: 0.0000e+00
Epoch 74/100
60/60 [==============================] - 0s 1ms/step - loss: 7.3471 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0333 - dense_1_acc_1: 0.5833 - dense_1

60/60 [==============================] - 0s 2ms/step - loss: 6.8160 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0333 - dense_1_acc_1: 0.6167 - dense_1_acc_2: 0.8833 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000 - dense_1_acc_16: 1.0000 - dense_1_acc_17: 1.0000 - dense_1_acc_18: 1.0000 - dense_1_acc_19: 1.0000 - dense_1_acc_20: 1.0000 - dense_1_acc_21: 1.0000 - dense_1_acc_22: 1.0000 - dense_1_acc_23: 1.0000 - dense_1_acc_24: 1.0000 - dense_1_acc_25: 1.0000 - dense_1_acc_26: 1.0000 - dense_1_acc_27: 1.0000 - dense_1_acc_28: 1.0000 - dense_1_acc_29: 0.0000e+00
Epoch 83/100
60/60 [==============================] - 0s 1ms/step - loss: 6.7586 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0333 - dense_1_acc_1: 0.6167 - dense_1

60/60 [==============================] - 0s 1ms/step - loss: 6.3836 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0667 - dense_1_acc_1: 0.6167 - dense_1_acc_2: 0.9000 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000 - dense_1_acc_16: 1.0000 - dense_1_acc_17: 1.0000 - dense_1_acc_18: 1.0000 - dense_1_acc_19: 1.0000 - dense_1_acc_20: 1.0000 - dense_1_acc_21: 1.0000 - dense_1_acc_22: 1.0000 - dense_1_acc_23: 1.0000 - dense_1_acc_24: 1.0000 - dense_1_acc_25: 1.0000 - dense_1_acc_26: 1.0000 - dense_1_acc_27: 1.0000 - dense_1_acc_28: 1.0000 - dense_1_acc_29: 0.0000e+00
Epoch 92/100
60/60 [==============================] - 0s 1ms/step - loss: 6.3421 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0667 - dense_1_acc_1: 0.6167 - dense_1

60/60 [==============================] - 0s 1ms/step - loss: 6.0612 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0667 - dense_1_acc_1: 0.6500 - dense_1_acc_2: 0.9167 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 1.0000 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000 - dense_1_acc_16: 1.0000 - dense_1_acc_17: 1.0000 - dense_1_acc_18: 1.0000 - dense_1_acc_19: 1.0000 - dense_1_acc_20: 1.0000 - dense_1_acc_21: 1.0000 - dense_1_acc_22: 1.0000 - dense_1_acc_23: 1.0000 - dense_1_acc_24: 1.0000 - dense_1_acc_25: 1.0000 - dense_1_acc_26: 1.0000 - dense_1_acc_27: 1.0000 - dense_1_acc_28: 1.0000 - dense_1_acc_29: 0.0000e+00


In [18]:
def music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, number of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    # Step 1: Create an empty list of "outputs" to later store your predicted values 
    outputs = []
    
    # Loop over Ty and generate a value at every time step
    for t in range(Ty):
        
        # Perform one step of LSTM_cell 
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        
        # Apply Dense layer to the hidden state output of the LSTM_cell 
        out = densor(a)

        # Append the prediction "out" to "outputs". out.shape = (None, 78)
        outputs.append(out)
         
        # Set "x" to be the one-hot representation of the selected value according to "out"
        x = Lambda(one_hot)(out)
        
    # Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = Model(inputs=[x0, a0, c0], outputs=outputs)
    
    return inference_model

In [19]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 50)

In [20]:
inference_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1, 78)        0                                            
__________________________________________________________________________________________________
a0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
c0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 64), (None,  36608       input_2[0][0]                    
                                                                 a0[0][0]                         
          

In [21]:
# Initialize Inference Model
x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

In [22]:
def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, 78), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    # Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    # Convert "pred" into an np.array() of indices with the maximum probabilities
    indices = np.argmax(pred, axis=-1)
    # Convert indices to one-hot vectors, the shape of the results should be (Ty, n_values)
    results = to_categorical(indices, num_classes=78)
    
    return results, indices

In [23]:
results, indices = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)
print("np.argmax(results[12]) =", np.argmax(results[12]))
print("np.argmax(results[17]) =", np.argmax(results[17]))
print("list(indices[12:18]) =", list(indices[12:18]))

np.argmax(results[12]) = 50
np.argmax(results[17]) = 46
list(indices[12:18]) = [array([50], dtype=int64), array([75], dtype=int64), array([45], dtype=int64), array([8], dtype=int64), array([44], dtype=int64), array([46], dtype=int64)]


In [24]:
out_stream = generate_music(inference_model)

Predicting new values for different set of chords.
Generated 50 sounds using the predicted values for the set of chords ("1") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("2") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("3") and after pruning
Generated 50 sounds using the predicted values for the set of chords ("4") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("5") and after pruning
Your generated music is saved in output/my_music.midi


In [ ]:
IPython.display.Audio('./output/my_music.midi')